In [1]:
from __future__ import print_function

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop, Adadelta, Adagrad, Adam, Nadam, SGD
from keras.callbacks import EarlyStopping, TerminateOnNaN
from keras import backend as K
from keras.losses import mean_squared_error
from keras.models import load_model, Model
import tensorflow as tf

# Normalize the data.
from sklearn import preprocessing
from keras.regularizers import l1_l2

import random

def pass_arg(nsim, tr_size):
    print("Tr_size:", tr_size)
    def fix_seeds(seed):
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
        session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
        sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
    #     K.set_session(sess)
        tf.compat.v1.keras.backend.set_session(sess)

    ss = 1
    fix_seeds(ss)

    # MC dropout
    class MCDropout(Dropout):
        def call(self, inputs, training=None):
            return super(MCDropout, self).call(inputs, training=True)

    # import pickle

    # def save_obj(obj, name):
    #     with open(name, 'wb') as f:
    #         pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

    
    # Compute the RMSE given the ground truth (y_true) and the predictions(y_pred)
    def root_mean_squared_error(y_true, y_pred):
            return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 

    # Making sure dimensionless bond length is less than 1
    def bond(bl):
        return tf.add(K.relu(tf.negative(bl)), K.relu(bl-1.0))

    # Making sure final porosity is less than initial
    def poros(poroi, porof):
        return K.relu(tf.negative(porof)) + K.relu(porof-poroi)

    def strength1(bl, porof, nlayer=6):
        sigma01, sigma02, C1s = 6, 31, 21
        sigma_long = sigma01*(K.exp((1.0-porof)**(C1s*nlayer))-porof) + sigma02*(1.0-porof)
        n = K.shape(sigma_long)[0]  
        sorted_strength, sortedIndices = tf.math.top_k(sigma_long, n, True)
        sorted_bl = K.gather(bl, sortedIndices)
        sorted_porof = K.gather(porof, sortedIndices)
        argg = tf.argsort(sorted_bl,axis=-1,direction='DESCENDING',stable=False,name=None)
        sorted_bl_corr = K.gather(sorted_bl, argg)
        return sorted_bl_corr-sorted_bl


    def strength2(bl, porof, nlayer=6):
        sigma01, sigma02, C1s = 6, 31, 21
        sigma_long = sigma01*(K.exp((1.0-porof)**(C1s*nlayer))-porof) + sigma02*(1.0-porof)
        n = K.shape(sigma_long)[0]  
        sorted_strength, sortedIndices = tf.math.top_k(sigma_long, n, True)
        sorted_bl = K.gather(bl, sortedIndices)
        n = K.cast(n, tf.float32)
        rel = K.relu(sorted_bl[1:]-sorted_bl[0:-1])
        num_vio = K.cast(tf.math.count_nonzero(rel), tf.float32)
        return num_vio/n


    def phy_loss_mean(params):
        # useful for cross-checking training
        loss1, loss2, loss3, loss4, lam1, lam2, lam3, lam4 = params
        def loss(y_true,y_pred):
    #         return lam1*K.mean(K.relu(loss1)) + lam2*K.mean(K.relu(loss2)) + lam2*K.mean(K.relu(loss3))
            return lam1*K.mean(K.relu(loss1)) + lam2*K.mean(K.relu(loss2)) + lam3*K.mean(K.relu(loss3)) + lam4*loss4
        return loss

    #function to calculate the combined loss = sum of rmse and phy based loss
    def combined_loss(params):
        loss1, loss2, loss3, loss4, lam1, lam2, lam3, lam4 = params
        def loss(y_true,y_pred):
    #         return mean_squared_error(y_true, y_pred) + lam1 * K.mean(K.relu(loss1)) + lam2 * K.mean(K.relu(loss2)) + lam2 * K.mean(K.relu(loss3))
            return mean_squared_error(y_true, y_pred) + lam1 * K.mean(K.relu(loss1)) + lam2 * K.mean(K.relu(loss2)) + lam3*K.mean(K.relu(loss3)) + lam4 * loss4
        return loss

    def PGNN_train_test(optimizer_name, optimizer_val, use_YPhy, pre_train, tr_size, lamda, iteration, n_nodes, n_layers, drop_frac, reg, samp):

    #     fix_seeds(ss)

        # Hyper-parameters of the training process
    #     batch_size = tr_size
        batch_size = 1
        num_epochs = 50
        val_frac = 0.2
        patience_val = 50

        # Initializing results filename
        exp_name = "fine-tuned_" + pre_train + optimizer_name + '_usePhy' + str(use_YPhy) + '_trsize' + str(tr_size) + '_lamda' + str(lamda) + '_iter' + str(iteration)
        exp_name = exp_name.replace('.','pt')
        results_dir = '../results/'
        model_name = results_dir + exp_name + '_model.h5' # storing the trained model

        if reg==True and samp==25:
            results_name = results_dir + exp_name + '_results_25_regularizer.dat' # storing the results of the model
        elif reg==False and samp==25:
            results_name = results_dir + exp_name + '_results_25.dat' # storing the results of the model
        elif reg==True and samp==1519:
            results_name = results_dir + exp_name + '_results_1519_regularizer.dat' # storing the results of the model
        elif reg==False and samp==1519:
            results_name = results_dir + exp_name + '_results_1519.dat' # storing the results of the model

        # Load labeled data
        data = np.loadtxt('../data/labeled_data.dat')
    #     data = np.loadtxt('../data/labeled_data_BK_constw_unique.dat')
    #     data = np.loadtxt('../data/labeled_data_BK_constw_v2.dat')
#         x_labeled = data[:, :2] # -2 because we do not need porosity predictions
        x_label = data[:, :-3] # -2 because we do not need porosity predictions
        x_labeled = np.hstack((x_label[:,:2],x_label[:,-2:]))
        y_labeled = data[:, -3:-1] # dimensionless bond length and porosity measurements
        if samp==25:
            data = np.loadtxt('../data/unlabeled_data_BK_constw_v2_25.dat')
            x_unlabeled = data[:, :]
        elif samp==1519:
            data = np.loadtxt('../data/unlabeled_data_BK_constw_v2_1525.dat')
            x_unlabeled = data[:, :]

        x_unlabeled1 = x_unlabeled[:1303, :]
        x_unlabeled2 = x_unlabeled[-6:, :]
        x_unlabeled = np.vstack((x_unlabeled1,x_unlabeled2))

        # initial porosity
        init_poro = x_unlabeled[:, -1]
        x_unlabeled = np.hstack((x_unlabeled[:,:2],x_unlabeled[:,-3:-1]))
#         x_unlabeled = x_unlabeled[:, :2]


        # normalize dataset with MinMaxScaler
        scaler = preprocessing.MinMaxScaler(feature_range=(0.0, 1.0))
    #     scaler = preprocessing.StandardScaler()
        x_labeled = scaler.fit_transform(x_labeled)
        x_unlabeled = scaler.fit_transform(x_unlabeled)
#         y_labeled = scaler.fit_transform(y_labeled)

    #     # initial porosity & physics outputs are removed
    #     x_unlabeled = x_unlabeled[:, :-3]

        # train and test data
        trainX, trainY = x_labeled[:tr_size,:], y_labeled[:tr_size]
    #     testX, testY = x_labeled[tr_size:,:], y_labeled[tr_size:]   
        testX, testY = x_labeled[tr_size:,:], y_labeled[tr_size:]

        if use_YPhy == 0:
            # Removing the last column from x_unlabeled (corresponding to Y_PHY)
            x_unlabeled = x_unlabeled[:,:-1]

        dependencies = {'root_mean_squared_error': root_mean_squared_error}

        # load the pre-trained model using non-calibrated physics-based model predictions (./data/unlabeled.dat)
        loaded_model = load_model(results_dir + pre_train, custom_objects=dependencies)

        # Creating the model
        model = Sequential()
        for layer in np.arange(n_layers):
            if layer == 0:
                model.add(Dense(n_nodes, activation='relu', input_shape=(np.shape(trainX)[1],)))
            else:
                if reg:
                    model.add(Dense(n_nodes, activation='relu', kernel_regularizer=l1_l2(l1=.001, l2=.001)))
                else:
                    model.add(Dense(n_nodes, activation='relu'))
            # model.add(Dropout(rate=drop_frac))
            model.add(MCDropout(rate=drop_frac))
        model.add(Dense(2, activation='linear'))
		
		# pass the weights to all layers but 1st input layer, whose dimensions are updated
        for new_layer, layer in zip(model.layers[1:], loaded_model.layers[1:]):
            new_layer.set_weights(layer.get_weights())

        # physics-based regularization
        uinp_sc = K.constant(value=x_unlabeled) # unlabeled input data
        lam1 = K.constant(value=lamda[0]) # regularization hyper-parameter
        lam2 = K.constant(value=lamda[1]) # regularization hyper-parameter
        lam3 = K.constant(value=lamda[2]) # regularization hyper-parameter
        lam4 = K.constant(value=lamda[3]) # regularization hyper-parameter
        predictions = model(uinp_sc) # model output at depth i
    #     porosity = K.relu(predictions[:,1])
        phyloss1 = bond(predictions[:,0]) # physics loss 1
    #     uinp = K.constant(value=x_unlabeled_non) # unlabeled input data
        phyloss2 = poros(init_poro, predictions[:,1]) # physics loss 1
        phyloss3 = strength1(predictions[:,0], predictions[:,1])
        phyloss4 = strength2(predictions[:,0], predictions[:,1])
        totloss = combined_loss([phyloss1, phyloss2, phyloss3, phyloss4, lam1, lam2, lam3, lam4])
        phyloss = phy_loss_mean([phyloss1, phyloss2, phyloss3, phyloss4, lam1, lam2, lam3, lam4])


        model.compile(loss=totloss,
                      optimizer=optimizer_val,
                      metrics=[phyloss, root_mean_squared_error])

        early_stopping = EarlyStopping(monitor='val_loss', patience=patience_val, verbose=1)

    #     print('Running...' + optimizer_name)
        history = model.fit(trainX, trainY,
                            batch_size=batch_size,
                            epochs=num_epochs,
                            verbose=0,
                            validation_split=val_frac, callbacks=[early_stopping, TerminateOnNaN()])
    
#     early_stopping = EarlyStopping(monitor='loss', patience=patience_val, verbose=1)
#     history = model.fit(trainX, trainY,
#                         batch_size=batch_size,
#                         epochs=num_epochs,
#                         verbose=1,
#                         callbacks=[early_stopping, TerminateOnNaN()])

#     test_score = model.evaluate(testX, testY, verbose=0)
#     predictions = model.predict(x_labeled) # model output at depth i
#     print(np.sort(predictions[:,0], axis=0))
    
#     predictions = model.predict(x_unlabeled) # model output at depth i
#     print(np.sort(predictions[:,0], axis=0))
    
#     print('iter: ' + str(iteration) + ' useYPhy: ' + str(use_YPhy) + 
#           ' nL: ' + str(n_layers) + ' nN: ' + str(n_nodes) + 
#           ' lamda1: ' + str(lamda[0]) + ' lamda2: ' + str(lamda[1]) + ' trsize: ' + str(tr_size) + 
#           ' TestRMSE: ' + str(test_score[2]) + ' PhyLoss: ' + str(test_score[1]), ' TestLoss: ' + str(test_score[0]), "\n")

# #     print('iter: ' + str(iteration) + ' TestRMSE: ' + str(test_score[2]) + ' PhyLoss: ' + str(test_score[1]), "\n")

    
# #     model.save(model_name)
    
#     # save results
#     results = {'train_loss_1':history.history['loss_1'], 
#                                 'val_loss_1':history.history['val_loss_1'], 
#                                 'train_rmse':history.history['root_mean_squared_error'], 
#                                 'val_rmse':history.history['val_root_mean_squared_error'],
#                                 'test_rmse':test_score[2],
#                                 'PhyLoss':test_score[1]}

#     results = {'train_loss_1':history.history['loss_1'], 
#                                 'train_rmse':history.history['root_mean_squared_error'], 
#                                 'test_rmse':test_score[2],
#                                 'PhyLoss':test_score[1]}

    

#     save_obj(results, results_name)

#     predictions = model.predict(testX)
#     return results, results_name, predictions, testY, test_score[2], trainY
    
        test_score = model.evaluate(testX, testY, verbose=1)
        print(test_score)

        Xx = np.random.uniform(0,1,(1000,2))
        xx1 = np.ones((1000,2))
        Xx = np.hstack((Xx,xx1))
        
        samples = []
        for i in range(int(nsim)):
#             print("simulation num:",i)
            predictions = model.predict(Xx)
            predictions = predictions[:,1]
            samples.append(predictions)
        return np.array(samples), testY
    
    



    # Main Function
    if __name__ == '__main__':

        fix_seeds(1)

        # List of optimizers to choose from    
        optimizer_names = ['Adagrad', 'Adadelta', 'Adam', 'Nadam', 'RMSprop', 'SGD', 'NSGD']
        optimizer_vals = [Adagrad(clipnorm=1), Adadelta(clipnorm=1), Adam(clipnorm=1), Nadam(clipnorm=1), RMSprop(clipnorm=1), SGD(clipnorm=1.), SGD(clipnorm=1, nesterov=True)]

        # selecting the optimizer
        optimizer_num = 1
        optimizer_name = optimizer_names[optimizer_num]
        optimizer_val = optimizer_vals[optimizer_num]

        # Selecting Other Hyper-parameters
        drop_frac = 0.1 # Fraction of nodes to be dropped out
        use_YPhy = 1 # Whether YPhy is used as another feature in the NN model or not
        n_layers = 2 # Number of hidden layers
        n_nodes = 5 # Number of nodes per hidden layer

        #set lamda
        lamda = [0.3, 0.15, 0.008, 0] # Physics-based regularization constant  

#         # Iterating over different training fractions and splitting indices for train-test splits
#         trsize_range = [4,6,8,10,20]

#         #default training size = 5000
#         tr_size = trsize_range[4]
        
        tr_size = int(tr_size)
        pre_train = 'Pre-trainAdadelta_drop0_nL2_nN5_trsize1308_iter0.h5'

        # use regularizer
        reg = True

        # sample size used
        samp = 1519
    #     samp = 25

        # total number of runs
        iter_range = np.arange(1)
        testrmse=[]
        # iterating through all possible params
        for iteration in iter_range:
#             results, result_file, pred, obs, rmse, obs_train = PGNN_train_test(optimizer_name, optimizer_val, drop_frac, use_YPhy, 
#                             iteration, n_layers, n_nodes, tr_size, lamda, reg, samp)
#             testrmse.append(rmse)
            pred, obs = PGNN_train_test(optimizer_name, optimizer_val, use_YPhy, 
                                               pre_train, tr_size, lamda, iteration, n_nodes, n_layers, drop_frac, reg, samp)
            

    return np.squeeze(pred), obs

Using TensorFlow backend.


In [2]:
pred, obs = pass_arg(50, 20)
mc_pred=np.mean(pred,axis=0)

Tr_size: 20
19/19 [==============================] - 0s 52us/step
[0.022604381665587425, 0.0013917512260377407, 0.10428124666213989]


In [3]:
# import matplotlib.pyplot as plt

# # Plot training & validation accuracy values
# fig, ax = plt.subplots()
# plt.plot(mc_pred, obs[:,1], 'd')
# plt.ylim((-0.01, 0.1))
# plt.xlim((-0.01, 0.1))
# ax.plot([-0.01,1],[-0.01,1], transform=ax.transAxes, color='k')
# plt.show()

In [4]:
mc_pred

array([0.05958226, 0.06915719, 0.05733197, 0.0398545 , 0.04122408,
       0.09252442, 0.03038587, 0.03382599, 0.0660064 , 0.04784298,
       0.04593347, 0.03304945, 0.04339615, 0.04290608, 0.03188588,
       0.06057683, 0.03654942, 0.02875252, 0.04120921, 0.08028256,
       0.04207018, 0.03816717, 0.0643486 , 0.03541319, 0.04029098,
       0.05851   , 0.06464598, 0.02462157, 0.03286206, 0.07815065,
       0.04120263, 0.05631733, 0.03737757, 0.03490303, 0.05098108,
       0.05251754, 0.02371311, 0.03224104, 0.04958144, 0.04693342,
       0.06497452, 0.06144594, 0.09044453, 0.05519437, 0.04809144,
       0.03877452, 0.057241  , 0.0752776 , 0.04399383, 0.06202547,
       0.02034918, 0.02462558, 0.04870855, 0.04823713, 0.04324883,
       0.06696029, 0.02283373, 0.02903545, 0.05433143, 0.04016906,
       0.06920074, 0.0400499 , 0.06008915, 0.05823291, 0.03438285,
       0.05087456, 0.01263563, 0.02946496, 0.05150693, 0.06455147,
       0.04615134, 0.02385854, 0.07156917, 0.08996201, 0.04686

In [5]:
obs[:,1]

array([0.01666471, 0.01358868, 0.05432798, 0.03850274, 0.01010751,
       0.01129286, 0.009357  , 0.03579652, 0.01614062, 0.00770211,
       0.0105933 , 0.03315482, 0.01352198, 0.05269549, 0.00490424,
       0.00895544, 0.01003564, 0.01345683, 0.02496436])

In [6]:
import pickle

def save_obj(obj, name):
    with open(name, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
        
save_obj(mc_pred, "../pred_upd_loss_hyb_MC_Xx.dat")